# Multi-agent adoption behaviour

### Imports

In [1]:
from Model.Code.src.models.EconomicModel import EconomicModel
from Model.Code.src.models.NetworkModel import NetworkModel
import numpy as _np
import pandas as _pd
from mesa.batchrunner import batch_run
import logging
from multiprocessing import freeze_support
logging.basicConfig(level=logging.CRITICAL)


### Experiments

In [2]:

if __name__ == '__main__':
    freeze_support()

    # for usage_trend in ["STABLE-LARGE", "UP-LARGE", "DOWN-LARGE"]:
    for price_trend in ["None", "VET-up", "VET-down"]:
        for other_user_strategy in ["RANDOM", "UNIFORM", "A-ADAPTED"]:

            # General network settings
            simulation_length = 7300
            generation_rate = 0.000432
            total_starting_VET = 86712634466.0  # 86,712,634,466
            total_starting_VTHO = 38396354542  # 38,396,354,542

            # Economic settings
            VET_starting_price = 0.0235
            VTHO_starting_price = 0.0015
            VET_liquidity_ratio = 0.00674
            VTHO_liquidity_ratio = 0.01226

            # if usage_trend == "STABLE-LARGE":
            #     _initial_multiplier = 0.6
            #     _final_multiplier = 0.6
            # elif usage_trend == "UP-LARGE":
            #     _initial_multiplier = 0.75
            #     _final_multiplier = 1
            # elif usage_trend == "DOWN-LARGE":
            #     _initial_multiplier = 0.75
            #     _final_multiplier = 0.5

            # Usage settings
            initial_VTHO_usage = total_starting_VET * generation_rate * 0.8
            final_VTHO_usage = total_starting_VET * generation_rate * 0.8
            small_user_size = initial_VTHO_usage / 4
            large_user_size = initial_VTHO_usage / 4

            economic_model = EconomicModel(
                economic_influences="None",
                price_trend_setting=price_trend,
                price_trend_length=simulation_length,
                steps_between_price_trend=simulation_length/365,
                VET_starting_price=VET_starting_price,
                VTHO_starting_price=VTHO_starting_price,
                total_starting_VET=total_starting_VET,
                total_starting_VTHO=total_starting_VTHO,
                VET_liquidity_ratio=VET_liquidity_ratio,
                VTHO_liquidity_ratio=VTHO_liquidity_ratio,
            )

            params = {
                "experiment_setting": ["MAS"],
                "economic_model": economic_model,
                "simulation_length": simulation_length,
                "generation_rate": generation_rate,
                "initial_VTHO_usage": initial_VTHO_usage,
                "final_VTHO_usage": final_VTHO_usage,
                "small_user_size": small_user_size,
                "large_user_size": large_user_size,
                "usage_trend": ["STABLE-LARGE"],
                "usage_trend_length": simulation_length,
                "starting_usage_trend_size": 0,
                "user_strategies": [other_user_strategy],
                "main_user_strategy": ["A-ADAPTED"],
            }
            # Gather the results
            batch_results = batch_run(model_cls=NetworkModel,
                                      parameters=params, iterations=10000, data_collection_period=-1, max_steps=simulation_length+1)

            # Convert the results to a DataFrame
            results_df = _pd.DataFrame(batch_results)

            # Export the results to csv
            results_df.to_csv(
                f"Model/Experiments/mas/output/data/10000_price_trend/MAS_a-adapted_{other_user_strategy}_{price_trend}.csv")

            for name in dir():
                # if not name.startswith("_"):
                if name in ["economic_model", "params", "batch_results", "results_df"]:
                    del globals()[name]


1it [00:21, 21.16s/it]
